# Demonstration Recording Script

This is a script for recording multiple demonstrations based on an environment.

`task_variant` can be set to `rR` (rotation reduced) or `rP` (rotation plus).

It has some special functions that allow re-orientation of the objects, this is done with the `update_object_orn` funciton, which gets and modifies the object pose.

In [1]:
import os
import time
import json
import shutil
import unittest
import subprocess
from pathlib import Path

from scipy.spatial.transform import Rotation as R

from gym_grasping.envs.robot_sim_env import RobotSimEnv
from flow_control.demo.demo_episode_recorder import record_sim
from flow_control.runner import evaluate_control
from flow_control.servoing.module import ServoingModule

root_dir = "/tmp/flow_experiments3"
renderer = "debug"
task="shape_sorting"
object_selected = "trapeze"
task_variant = "rP"  # rotation plus (+-pi)

def get_configurations(root_dir=root_dir, num_episodes=20, prefix=""):
    os.makedirs(root_dir, exist_ok=True)
    save_dir_template = os.path.join(root_dir, f"{prefix}_{task}_{object_selected}")
    for seed in range(num_episodes):
        save_dir = save_dir_template + f"_{task_variant}"+f"_seed{seed:03d}"
        yield object_selected, task_variant, seed, save_dir

pybullet build time: May 20 2022 19:44:17


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Record Episodes


In [2]:
from math import pi

# def update_object_orn(env, object_selected, new_orn):
#     object_uid = env._task.object_uids[object_selected]
#     object_pos, object_orn = env.p.getBasePositionAndOrientation(object_uid)
#     print("pos & orn", object_pos, object_orn)
    
#     env.p.resetBasePositionAndOrientation(object_uid, object_pos, new_orn,
#                                           physicsClientId=env.cid)

#     object_pos, object_orn = env.p.getBasePositionAndOrientation(object_uid)
#     print("pos & orn2", object_pos, object_orn)

demo_cfgs = get_configurations(prefix="demo")
for object_selected, task_variant, seed, save_dir in demo_cfgs:
    param_info = {"object_selected": object_selected}
    env = RobotSimEnv(task='shape_sorting', renderer=renderer, act_type='continuous',
                      initial_pose='close', max_steps=200, control='absolute-full',
                      img_size=(256, 256),
                      param_randomize=("geom",),
                      param_info=param_info,
                      task_info=dict(object_rot_range={"rP":pi/2.,"rR":pi/6.}[task_variant]),
                      seed=seed)
    
    if task_variant == "rP":
        assert env.params.variables[f"{object_selected}_pose"]["d"][3] == pi/2.
    elif task_variant == "rR":
        assert env.params.variables[f"{object_selected}_pose"]["d"][3] == pi/6.
    #update_object_orn(env, object_selected, orn)
        
    if os.path.isdir(save_dir):
        # lsof file if there are NSF issues.
        shutil.rmtree(save_dir)
    record_sim(env, save_dir)
    
    del env
    time.sleep(.5)
    print(save_dir)

/home/buechner/venvs/servo/lib/python3.8/site-packages/gym-0.26.2-py3.8.egg/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


TypeError: step() missing 1 required positional argument: 'info'

In [ ]:
# Convert notebook to script
convert_cmd = "jupyter nbconvert --to script Demonstration_Viewer.ipynb"
convert_cmd = convert_cmd.split()
subprocess.run(convert_cmd, check=True)

for _, _, seed, save_dir in get_configurations(prefix="demo"):
    segment_cmd = f"python ./Demonstration_Viewer.py {save_dir}"
    subprocess.run(segment_cmd.split(), check=True)

# Cleanup, don't leave file lying around because e.g. github PEP check
os.remove("./Demonstration_Viewer.py")

## Show Recorded Demos. 

In [ ]:
from flow_control.servoing.playback_env_servo import PlaybackEnvServo

demo_cfgs = get_configurations(prefix="demo")
recordings = []
for _, _, demo_seed, demo_dir in demo_cfgs:
    recordings.append(demo_dir)
    
print("Number of recordings:", len(recordings))
print(recordings[0])
print(recordings[-1])
# Load the demonstration episodes
playbacks = [PlaybackEnvServo(rec) for rec in recordings[:]]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact, Layout

# Plot the demonstrations
%matplotlib notebook
fig, ax = plt.subplots(1,figsize=(8, 6))
fig.suptitle("Demonstration Frames")
ax.set_axis_off()
image_h = ax.imshow(playbacks[0].cam.get_image()[0])

def update(demo_index, frame_index):
    image = playbacks[demo_index][frame_index].cam.get_image()[0]
    image_h.set_data(image)
    fig.canvas.draw_idle()
    print("wp_name:", playbacks[demo_index][frame_index].get_info()["wp_name"])
    print(playbacks[demo_index][frame_index].get_info()["move_anchor"])
    
    fg_mask = playbacks[demo_index].get_fg_mask()
    if fg_mask is not None:
        print("percent fg @ 0:", np.mean(fg_mask)*100)
    
slider_w = widgets.IntSlider(min=0, max=len(playbacks)-1, step=1, value=0,
                             layout=Layout(width='70%'))
slider_i = widgets.IntSlider(min=0, max=200-1, step=1, value=0,
                             layout=Layout(width='70%'))

interact(update, demo_index=slider_w, frame_index=slider_i)

In [ ]:
def filter_demo(pb):
    return pb[-1].data['rew'] > 0 and np.mean(pb.get_fg_mask()) > 0.005

demo_good = [filter_demo(pb) for pb in playbacks]
good_demonstrations = np.where(demo_good)[0]
print(good_demonstrations)
good_demonstrations = [int(x) for x in good_demonstrations]

#print(np.array(demo_good).astype(int))

## Segment Demonstration into Parts

This is done via the waypoint names, which have the following format `<block-name>_<detail-descr>`.
We group `block-name` together and apply block_name_map to group blocks.

In [ ]:
from itertools import tee
block_name_map, parts_name = {'start': 'locate'}, "manual3"
#block_name_map, parts_name = {'start': 'locate', 'grasp': 'locate'}, "manual2"

def get_block_name(wp_name, block_map):
    block_name = wp_name.split("_")[0]
    if block_name in block_map:
        return block_map[block_name]
    return block_name

def pairwise(iterable):
    # pairwise('ABCDEFG') --> AB BC CD DE EF FG
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

demo_parts = {}
for demo_index in good_demonstrations:
    wp_names = []
    for frame_index in range(len(playbacks[demo_index])-1):
        wp_name = playbacks[demo_index][frame_index].get_info()["wp_name"]
        wp_names.append(wp_name)
        
    block_names = np.array([get_block_name(wp, block_name_map) for wp in wp_names])
    transitions = np.where(block_names[1:] != block_names[:-1])[0]
    print(transitions)
    for i in range(len(transitions)+1):
        if i == 0:
            pass
    trn_name = [*block_names[transitions],  block_names[transitions[-1]+1]]
    trn_idx = pairwise([0,*transitions, len(playbacks[demo_index])-1])
    
    ep_demo_parts = []
    for name, trn in zip(trn_name, trn_idx):
        ep_demo_parts.append(dict(name=name, start=int(trn[0]), end=int(trn[1])))
    demo_parts[demo_index] = ep_demo_parts
    print(ep_demo_parts)

print(f"Demo Parts (demo={demo_index}):")
for e in ep_demo_parts:
    print(f"{e['name']}:\t{e['start']:02d}->{e['end']:02d}")

demo_parts_fn = os.path.join(root_dir, f"demo_parts_{parts_name}.json")
with open(demo_parts_fn, "w") as f_obj:
    json.dump(demo_parts, f_obj)
    
print("saved demo parts to :", demo_parts_fn)

In [ ]:
# Plot the end of segments
%matplotlib notebook
max_ep_parts = max([len(ep_parts) for ep_parts in demo_parts.values()])

fig, (ax, ax2) = plt.subplots(1,2,figsize=(8, 6))
fig.suptitle("Show Demonstration Parts")
ax.set_axis_off()
ax2.set_axis_off()
image_h = ax.imshow(playbacks[0].cam.get_image()[0])
image_h2 = ax2.imshow(playbacks[0].cam.get_image()[0])

max_frames = max([len(b) for b in demo_parts.values()])
def update(demo_ep, part_idx):
    demo_index = list(demo_parts.keys())[demo_ep]
    demo_name = demo_parts[demo_index][part_idx]["name"]
    start_index = demo_parts[demo_index][part_idx]["start"]
    end_index = demo_parts[demo_index][part_idx]["end"]
    ax.set_title(f"{demo_name}: start @ {demo_index}, {start_index}")
    ax2.set_title(f"{demo_name}: end @ {demo_index}, {end_index}")
    
    image = playbacks[demo_index][start_index].cam.get_image()[0]
    image_h.set_data(image)
    image2 = playbacks[demo_index][end_index].cam.get_image()[0]
    image_h2.set_data(image2)
    fig.canvas.draw_idle()

    print("wp_name:", playbacks[demo_index][start_index].get_info()["wp_name"])
    
slider_w = widgets.IntSlider(min=0, max=len(demo_parts)-1, step=1, value=0,
                             layout=Layout(width='70%'))

slider_i = widgets.IntSlider(min=0, max=max_ep_parts-1, step=1, value=0,
                             layout=Layout(width='70%'))

interact(update, demo_ep=slider_w, part_idx=slider_i)
plt.tight_layout()
plt.show()

### Check if the block size is still randomized